In [2]:

import json
import os
import pandas as pd
import pprint
import tensorflow as tf
import time
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import *

print(tf.__version__)


1.12.0


In [3]:

class ConvBnAct(tf.keras.Model):
  
    def __init__(self, n_filters=64, kernel=(2,2), strides=(1,1), padding='valid'):
        self.filters = n_filters
        self.kernel = kernel
        self.strides = strides
        self.activation = tf.nn.relu
        super(ConvBnAct, self).__init__(name='')
        
        #print('padding', padding)
        self.conv_ =  Conv2D(filters=self.filters,
                      kernel_size = self.kernel,
                      strides = self.strides, name='Conv2D', 
                      padding=padding)

        self.batch_norm = BatchNormalization(name='BatchNorm')

        self.activation = Activation(self.activation, name='RELU')
    
    def call(self, x, training=False):
        x = self.conv_(x) 
        x = self.batch_norm(x)

        x = self.activation(x)

        return x

In [4]:
norm = tf.random_normal([10, 256, 256, 64], mean=-1, stddev=4)

block = ConvBnAct()
block(norm)
print([x.name for x in block.variables])


['conv_bn_act/Conv2D/kernel:0', 'conv_bn_act/Conv2D/bias:0', 'conv_bn_act/BatchNorm/gamma:0', 'conv_bn_act/BatchNorm/beta:0', 'conv_bn_act/BatchNorm/moving_mean:0', 'conv_bn_act/BatchNorm/moving_variance:0']


In [5]:
class ConvAct(tf.keras.Model):
    def __init__(self, n_filters, kernel=(1,1), activation = tf.nn.relu, pooling=False):
      super(ConvAct, self).__init__(name='')

      self.pooling = pooling
      self.kernel = kernel
      self.n_filters = n_filters
      self.activation = activation
      self.pooling = pooling


      self.poolingLayer = AveragePooling2D(pool_size=(1,1), padding='same', name='AveragePool')
      self.convLayer = Conv2D(filters = self.n_filters,
                         kernel_size = self.kernel,
                         strides=1, name='Conv2D')

      self.activation = Activation(activation, name='ActivationRelu')

    def call(self, x, training = False):

      if self.pooling:
        x = self.poolingLayer(x)

      x = self.convLayer(x)
      x = self.activation(x)

      return x

In [6]:

norm = tf.random_normal([10, 127, 127, 64], mean=-1, stddev=4)

block = ConvAct(n_filters=64, pooling=True)
block(norm)

print([x.name for x in block.variables])

['conv_act/Conv2D/kernel:0', 'conv_act/Conv2D/bias:0']


In [7]:
class AttentionRefinmentModule(tf.keras.Model):
  def __init__(self, n_filters):
    super(AttentionRefinmentModule, self).__init__(name='')

    self.filters = n_filters
    
    self.poolingLayer = AveragePooling2D(pool_size = (1,1), padding='same', name='AveragePool')
    self.conv_bn_act = ConvBnAct(kernel = (1,1), n_filters = self.filters)
    
    
  def call(self, inputs, training = False):
    x = self.poolingLayer(inputs)
    x = self.conv_bn_act(x)
    
    print(x.shape)
    print(inputs.shape)
    return tf.math.multiply(inputs,x)

In [8]:
norm = tf.random_normal([10, 256, 256, 64], mean=-1, stddev=4)

block = AttentionRefinmentModule(n_filters=64)
block(norm)

print([x.name for x in block.variables])

(10, 256, 256, 64)
(10, 256, 256, 64)
['attention_refinment_module/conv_bn_act_1/Conv2D/kernel:0', 'attention_refinment_module/conv_bn_act_1/Conv2D/bias:0', 'attention_refinment_module/conv_bn_act_1/BatchNorm/gamma:0', 'attention_refinment_module/conv_bn_act_1/BatchNorm/beta:0', 'attention_refinment_module/conv_bn_act_1/BatchNorm/moving_mean:0', 'attention_refinment_module/conv_bn_act_1/BatchNorm/moving_variance:0']


In [9]:

class FeatureFusionModule(tf.keras.Model):
  def __init__(self, n_filters):
    super(FeatureFusionModule, self).__init__(name='')
    self.conv_bn_act = ConvBnAct(n_filters=n_filters, kernel=(3, 3), padding='same')
    self.conv_act1 = ConvAct(n_filters=n_filters, pooling=True)
    self.conv_act2 = ConvAct(n_filters=n_filters, pooling=False, activation = tf.nn.sigmoid)
    activation = tf.nn.sigmoid
    
   
  def call(self, input_f, input_s, training=True):
    
    print(input_f.shape, input_s.shape)
    concate = Concatenate(axis=-1)([input_f, input_s])
    
    print(concate.shape)
    branch0 = self.conv_bn_act(concate)
    print(branch0.shape)
    branch_1 = self.conv_act1(branch0)
    
    branch_1 = self.conv_act2(branch_1)
    
    x = multiply([branch0, branch_1])
    return  Add()([branch0, x])


In [10]:
norm1 = tf.random_normal([10, 256, 256, 64], mean=-1, stddev=4)
norm2 = tf.random_normal([10, 256, 256, 64], mean=-1, stddev=4)

block = FeatureFusionModule(n_filters=64)
block(norm1, norm2)

print([x.name for x in block.variables])

norm1 = tf.random_normal([10, 256, 256, 64], mean=-1, stddev=4)
print(GlobalMaxPooling2D()(norm1).shape)

(10, 256, 256, 64) (10, 256, 256, 64)
(10, 256, 256, 128)
(10, 256, 256, 64)
['feature_fusion_module/conv_bn_act_2/Conv2D/kernel:0', 'feature_fusion_module/conv_bn_act_2/Conv2D/bias:0', 'feature_fusion_module/conv_bn_act_2/BatchNorm/gamma:0', 'feature_fusion_module/conv_bn_act_2/BatchNorm/beta:0', 'feature_fusion_module/conv_act_1/Conv2D/kernel:0', 'feature_fusion_module/conv_act_1/Conv2D/bias:0', 'feature_fusion_module/conv_act_2/Conv2D/kernel:0', 'feature_fusion_module/conv_act_2/Conv2D/bias:0', 'feature_fusion_module/conv_bn_act_2/BatchNorm/moving_mean:0', 'feature_fusion_module/conv_bn_act_2/BatchNorm/moving_variance:0']
(10, 64)


In [11]:

class ContextPath(tf.keras.Model):
  def __init__(self):
    super(ContextPath, self).__init__(name='')
    
    self.globalmax = GlobalAveragePooling2D()
    self.upsample_1 = UpSampling2D(size=(2, 2))
    self.upsample_2 = UpSampling2D(size=(2, 2))
    self.upsample_3 = UpSampling2D(size=(2, 2))
    self.upsample_4 = UpSampling2D(size=(2, 2))
    self.arm1 = AttentionRefinmentModule(n_filters=1024)
    self.arm2 = AttentionRefinmentModule(n_filters=2048)

  def call(self, layer_13, layer_14):
    globalmax = self.globalmax(layer_14)
    upsample1 = self.upsample_1(layer_14)
    first_ = Add()([globalmax, upsample1])
    
    seg1 = self.arm1(layer_13)
    seg2 = self.arm2(layer_14)
    
    up_3 = self.upsample_2(seg1)
    up_4 = self.upsample_3(seg2)

    
    cnc = Concatenate(axis=-1)([up_3, up_4])
    cnc = Concatenate(axis=-1)([cnc, first_])


    context_features = self.upsample_4(cnc)
    return context_features

In [12]:

layer_13 = tf.random_normal([10, 7, 7, 1024], mean=-1, stddev=4)
layer_14 = tf.random_normal([10, 7, 7, 2048], mean=-1, stddev=4)

block = ContextPath()
block(layer_13,layer_14)


(10, 7, 7, 1024)
(10, 7, 7, 1024)
(10, 7, 7, 2048)
(10, 7, 7, 2048)


<tf.Tensor 'context_path/up_sampling2d_3/ResizeNearestNeighbor:0' shape=(10, 28, 28, 5120) dtype=float32>

In [13]:
class FinalModel(tf.keras.Model):
  def __init__(self):
    super(FinalModel, self).__init__(name='')
    self.context_path = ContextPath()
    self.batch1 = ConvBnAct(32, strides=2)
    self.batch2 = ConvBnAct(64, strides=2)
    self.batch3 = ConvBnAct(156, strides=2)
    self.upsample = UpSampling2D(size=(8, 8))
    self.feature_fusion = FeatureFusionModule(32)
    
  def call(self, x, layer_13, layer_14): 
    
    #spatial path
    x = self.batch1(x)
    x = self.batch2(x)
    x = self.batch3(x)
    
    #context path
    cp = self.context_path(layer_13, layer_14)
    
    #concatenate
    print(cp.shape, x.shape, 'context')
    
    #upsample
    print('fusion')
    fusion = self.feature_fusion(cp, x, 32)
    print('fusion')
    ans = self.upsample(fusion)
    
    
    return ans


In [14]:
from keras.applications import *

Xception_model = Xception(weights='imagenet',input_shape= (224,224,3), include_top=False)

inputs = Input(shape=(224,224,3))
x = Lambda(lambda image: tf.image.resize_images(image, (224, 224)))(inputs)


Using TensorFlow backend.


In [15]:
layer_13 = Xception_model.get_layer('block13_pool').output
layer_14 = Xception_model.output

print(layer_13.shape)
print(layer_14.shape)

(?, 7, 7, 1024)
(?, 7, 7, 2048)


In [16]:
# # 16x Down
layer_13 = Xception_model.get_layer('block13_pool').output
layer_14 = Xception_model.output


In [17]:
# for layer in model.layers[:5]:
#     layer.trainable = False
input_layer = tf.random_normal([10, 224, 224, 3], mean=-1, stddev=4)

layer_13 = tf.random_normal([10, 7, 7, 1024], mean=-1, stddev=4)
layer_14 = tf.random_normal([10, 7, 7, 2048], mean=-1, stddev=4)

block = FinalModel()
t = block(input_layer, layer_13, layer_14)

print('fin',t.shape)

(10, 7, 7, 1024)
(10, 7, 7, 1024)
(10, 7, 7, 2048)
(10, 7, 7, 2048)
(10, 28, 28, 5120) (10, 28, 28, 156) context
fusion
(10, 28, 28, 5120) (10, 28, 28, 156)
(10, 28, 28, 5276)
(10, 28, 28, 32)
fusion
fin (10, 224, 224, 32)


In [25]:
from keras.applications.xception import Xception,preprocess_input
from keras.utils import to_categorical


In [72]:
import os 
import numpy as np 
from keras.preprocessing import image
import cv2

PATH = os.getcwd()
train_path = PATH+'\data\\train\\'
trainy_path = PATH+'\data\\train_labels\\'

print(train_path)

train_batch = os.listdir(train_path)
trainy_batch = os.listdir(trainy_path)

x_train = []
y_train = []


# if data are in form of images
for sample in train_batch:
    img_path = train_path+sample
    x = cv2.imread(img_path)
    # preprocessing if required
    x_train.append(x)


for sample in trainy_batch:
    img_path = trainy_path+sample
    x = cv2.imread(img_path)
    # preprocessing if required
    y_train.append(x)
    



C:\Users\Kiko-PC\Documents\Image-Segmentation\data\train\


In [32]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [64]:

def get_label_info(csv_path):
    """
    Retrieve the class names and label values for the selected dataset.
    Must be in CSV format!

    # Arguments
        csv_path: The file path of the class dictionairy
        
    # Returns
        Two lists: one for the class names and the other for the label values
    """
    filename, file_extension = os.path.splitext(csv_path)
    if not file_extension == ".csv":
        return ValueError("File is not a CSV!")

    class_names = []
    label_values = []
    with open(csv_path, 'r') as csvfile:
        file_reader = csv.reader(csvfile, delimiter=',')
        header = next(file_reader)
        for row in file_reader:
            class_names.append(row[0])
            label_values.append([int(row[1]), int(row[2]), int(row[3])])
        # print(class_dict)
    return class_names, label_values

def one_hot_it(label, label_values):
    """
    Convert a segmentation image label array to one-hot format
    by replacing each pixel value with a vector of length num_classes

    # Arguments
        label: The 2D array segmentation image label
        label_values
        
    # Returns
        A 2D array with the same width and hieght as the input, but
        with a depth size of num_classes
    """
    # st = time.time()
    # w = label.shape[0]
    # h = label.shape[1]
    # num_classes = len(class_dict)
    # x = np.zeros([w,h,num_classes])
    # unique_labels = sortedlist((class_dict.values()))
    # for i in range(0, w):
    #     for j in range(0, h):
    #         index = unique_labels.index(list(label[i][j][:]))
    #         x[i,j,index]=1
    # print("Time 1 = ", time.time() - st)

    # st = time.time()
    # https://stackoverflow.com/questions/46903885/map-rgb-semantic-maps-to-one-hot-encodings-and-vice-versa-in-tensorflow
    # https://stackoverflow.com/questions/14859458/how-to-check-if-all-values-in-the-columns-of-a-numpy-matrix-are-the-same
    semantic_map = []
    for colour in label_values:
        # colour_map = np.full((label.shape[0], label.shape[1], label.shape[2]), colour, dtype=int)
        equality = np.equal(label, colour)
        class_map = np.all(equality, axis = -1)
        semantic_map.append(class_map)
    semantic_map = np.stack(semantic_map, axis=-1)
    # print("Time 2 = ", time.time() - st)

    return semantic_map
    
def reverse_one_hot(image):
    """
    Transform a 2D array in one-hot format (depth is num_classes),
    to a 2D array with only 1 channel, where each pixel value is
    the classified class key.

    # Arguments
        image: The one-hot format image 
        
    # Returns
        A 2D array with the same width and hieght as the input, but
        with a depth size of 1, where each pixel value is the classified 
        class key.
    """
    # w = image.shape[0]
    # h = image.shape[1]
    # x = np.zeros([w,h,1])

    # for i in range(0, w):
    #     for j in range(0, h):
    #         index, value = max(enumerate(image[i, j, :]), key=operator.itemgetter(1))
    #         x[i, j] = index

    x = np.argmax(image, axis = -1)
    return x


In [69]:
import csv
class_names_list, label_values =get_label_info("data/class_dict.csv")
class_names_string = ""
for class_name in class_names_list:
    if not class_name == class_names_list[-1]:
        class_names_string = class_names_string + class_name + ", "
    else:
        class_names_string = class_names_string + class_name

num_classes = len(label_values)

In [70]:
print(num_classes)

32


In [ ]:
output_image = np.float32(one_hot_it(y_train, label_values=label_values))
print(output_image.shape)